# Sending utterances and generating logs

This notebook demonstrates a technique to programmatically Send utterances and generate conversation logs for an assistant in <a href="https://cloud.ibm.com/apidocs/assistant-v2" target="_blank" rel="noopener noreferrer">Watson Assistant</a>.

Sometimes you need a quick way to simulate a user and call the /message API with a sequence of input utterances. For example, you may want to test your workspace and generate data for the Improve section of the Conversation UI. you are going  to use this notebook with the sample utterances-for-generate-chat-logs CSV file to do this. 

This notebook will demonstrate how the Watson Assistant API can be directly accessed to programmatically send user examples. This is an alternative to the user GUI.

This notebook runs on Python 3.6
So, on the top right be sure to run the right Kernel, if not go to menu **Kernel > Change Kernel** then select **Python 3.6** 

**Tips:**
* Code cells are identifiable by their `In [ ]:` prefix in the margin
* To execute the celsl in the notebook, select the cell and click the run button, or hit Ctrl-Enter.
* Cells which have not been executed before will have empty brackets, while executed cells will have a sequence number within, e.g. `In [13]`
* Cell execution result displays below the cell
* To clear all exection statuses and outputs, use the `Cell/All Output/Clear` menu.

Then execute the cell (Ctrl-Enter or run button)

## Table of contents

1. [Install and import packages](#setup)
2. [Import the data as a pandas DataFrame](#import)
3. [Authenticate to the Watson Assistant Service](#authenticate)
4. [Test the connection to the Watson Assistant](#wcs1)
5. [Evaluate the test set with the message function](#wcs4)<br>
[Summary and next steps](#Summary-and-next-steps)

## <a id="setup"></a> Step 1. Install and import packages

Install and import the necessary packages.


In [ ]:
!pip install --upgrade "ibm-watson>=5.0.0"
#!pip install --upgrade ibm-watson

In [ ]:
import pandas as pd
import numpy as np
#from bokeh.charts import Histogram, output_file, show
import random
#from sklearn.model_selection import train_test_split

## <a id="import"></a>Step 2. Import the data as a pandas DataFrame

## 2.1 import the access Token

In order to read files this notebook you would need to create an access token. This project token must be set in notebooks so that project and platform functions can access the project resources.

you should have created an access token: if not, open the project main page in a new browser window, and create an access token by clicking on "Settings" tab and scroll down; you will see 'Access token' section and click on "New Token". Only project admins can create access tokens. Give the token the Editor access role.

you need to import the access token:

    1. Click on the "more option" (3 dots) icon on the top right and choose Insert Project Token.
    2. This action will create a code cell in this notebook. (Maybe on the top of the page). It should be something like this:

         from project_lib import Project
         project = Project(project_id='xxx', project_access_token='xxx')
         pc = project.project_context

    3. Copy and paste the content in the cell below (for tidyness) or just run the cell that was created.
    4. Run the cell


In [ ]:
### COPY AND PASTE HERE THE CONTENT OF "Insert Project Token" Code cell and run this cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
###


## 2.1 import Data Frame

The data consists of sample user questions and the assigned intents. 

**For notebooks running on IBM Data Science Experience:**

To get the data and load it into a pandas DataFrame:

**File description**
Given a CSV file containing utterances to simulate a user's questions, and a set of input parameters, this script will call the Watson Conversation /message API and simulate an interaction between a user and the Conversation service. The result will include data in the Improve tab of the Watson Conversation Service UI. The CSV file format allows you to include a sequence of utterances in the same conversation. Place each utterance on a separate line. To start a new conversation, you should include a line with the string :init:
For example, a CSV to simulate two conversations with the Car Dashboard Demo sample application # could be as simple as these six lines:
question
hi
turn lights on
:init:
hi
hit the brake

In [ ]:
### USER INPUT
# Change the name of the file
test_file = 'utterances-for-generate-chat-logs.csv'

## Fetch the file 
body = project.get_file(test_file)

##read the CSV data from file
body.seek(0)

df_data_1 = pd.read_csv(body)
df_data_1.head()

# Rename the DataFrame to test.
try:
    test = df_data_1
except NameError as e:
    print('Error: Setup is incorrect or incomplete.\n')
    raise
test.head()

## <a id="authenticate"></a>Step 3. Authenticate to the Watson Assistant

Sign up for the Watson Conversation service and enter your credentials. 

1. Sign up for [Watson Assistant](https://cloud.ibm.com/catalog/services/conversation) in IBM Cloud.
1. On your Watson Assistant service page, click **Launch Watson Assistant**. The Assistant and skills appear in a separate tab.
1. Select your Assistant which simulates your Production environment, Click on the 3 dots menu to select **Settings** option. 
1. On your Assistant settings page, select **API Details** tab and find your Assistant ID and API Key.
1. Your Service URL should be the root of the Assistant URL. The URL for Dallas/US South region looks like this https://api.us-south.assistant.watson.cloud.ibm.com/instances/instanceID
1. Add your Assistant ID, APIKey, Service URL to the next cell and run the cell.

Tips:
* The Watson Studio and your Watson services must be in the same IBM Cloud region (US South for instance)

In [ ]:
ASSISTANT_ID = '<Assistant ID>'
ASSISTANT_APIKEY = '<Assistant APIKey>'
SERVICE_URL = '<Service URL>'

#Frankfurt https://api.eu-de.assistant.watson.cloud.ibm.com/instances/instanceID
#Dallas https://api.us-south.assistant.watson.cloud.ibm.com/instances/instanceID
#London https://api.eu-gb.assistant.watson.cloud.ibm.com/instances/instanceID
#Sydney https://api.au-syd.assistant.watson.cloud.ibm.com/instances/instanceID
#Washington DC https://api.us-east.assistant.watson.cloud.ibm.com/instances/instanceID
#Tokyo https://api.jp-tok.assistant.watson.cloud.ibm.com/instances/instanceID
#Seoul https://api.kr-seo.assistant.watson.cloud.ibm.com/instances/instanceID

VERSION = '2021-06-14'
USER_ID='usertest'
CUSTOMER_ID=''

Import the Watson Assistant package and set variables:

In [ ]:
import json
import ibm_watson
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator(ASSISTANT_APIKEY)
assistant = AssistantV2(
    version=VERSION,
    authenticator=authenticator
)
assistant.set_service_url(SERVICE_URL)


## <a id="wcs1"></a>Step 4. Test the connection to the Watson Assistant
Run the <a href="https://www.ibm.com/watson/developercloud/assistant/api/v1/" target="_blank" rel="noopener noreferrer">Watson Assistant API</a> functions to make sure you are properly connected to your Watson Assistant.

Create a session and return the Session ID then delete the session.

In [ ]:
response = assistant.create_session(ASSISTANT_ID).get_result()
print('session ID : %s' % response['session_id'])
response = assistant.delete_session(ASSISTANT_ID,response['session_id']).get_result()

## <a id="wcs4"></a>Step 5. Evaluate the test set with the message function
    Now send user examples. By using the `message` function from the <a href="https://www.ibm.com/watson/developercloud/assistant/api/v1/" target="_blank" rel="noopener noreferrer">Watson Assistant API</a>, you can test all examples at once, instead of examining each example individually with the Assistant Workspace tool.

In [ ]:
from __future__ import print_function  # Python 3 support

import sys
import datetime
import re
import csv
import argparse

#2. Header setting if provided ###################################################
if CUSTOMER_ID is not None:
    assistant.set_default_headers({'X-Watson-Metadata': 'customer_id=' + CUSTOMER_ID})

#2. Built the message metadata and context if provided ##########################
""" 
Builds the context if user_id are specified as arguments
"""
mycontext = None
globalcontext = None
user_id = { "user_id": USER_ID} if USER_ID !='' else None
if user_id is not None:
    globalcontext = {'system':user_id} if user_id is not None else None
    mycontext = {'global': globalcontext} if globalcontext is not None else None
#    print(json.dumps(mycontext, indent=2))  

# 3. Read the CSV file#############################################################
#  Done previously


# 4. Process the csv file ##########################################################
    
test_result = True
turn_count = 0
session = assistant.create_session(ASSISTANT_ID).get_result()
#print('start session : %s' % session['session_id'])    
    
#    for turn in csv_reader:
for turn in [x[1] for x in test[:].iterrows()]:    
    turn_count += 1
    if turn['question'][0] == ';':
        continue
    if turn['question'] == ':init:':
        if turn_count > 1:
            endsession = assistant.delete_session(ASSISTANT_ID,session['session_id']).get_result()
#            print('end session : %s' % session['session_id'])
            mycontext = {'global': globalcontext} if globalcontext is not None else None
        session = assistant.create_session(ASSISTANT_ID).get_result()
#        print('start session : %s' % session['session_id'])
        turn_count = 0
        continue
#    print('Input : %s' % turn['question'])
#    print(json.dumps(mycontext, indent=2))
    
    response = None
    try:        
        #response = assistant.message(assistant_id=ASSISTANT_ID,session_id=session['session_id'],input={'message_type':'text','text': turn['question']}).get_result()
        #with context
        if mycontext is not None:
            response = assistant.message(assistant_id=ASSISTANT_ID,session_id=session['session_id'],input={'message_type':'text','text': turn['question'],'options':{'return_context': True}},context=mycontext).get_result()
        else:
        #without context
            response = assistant.message(assistant_id=ASSISTANT_ID,session_id=session['session_id'],input={'message_type':'text','text': turn['question'],'options':{'return_context': True}}).get_result()
    except WatsonApiException as error:
        print(error)
        sys.exit(1)
        
#    print(json.dumps(response, indent=2))
#    print(json.dumps(response['context'], indent=2))
    if globalcontext is not None:
        mycontext ={'global': globalcontext,'skills':response['context']['skills']}                                                                                                    
#        print(json.dumps(mycontext, indent=2))

    for generic in response['output']['generic']:    
        try:
            output = ' ' + generic['text']
        except:
            output = ' ' + 'Disambiguation'
        continue
    
    if turn_count == 1:
        print('Session ID: %s (%s)' % (session['session_id'], str(datetime.datetime.now())))
        
    print('Input : %s' % turn['question'])
    print('Output: %s\n' % output)


## Summary and next steps
You've learned how to use the Watson Assistant API to send user examples. Try adding your own user questions and see how Watson does!
you must review the 'improve' feature of Watson assistant to see what were captured by the service.

Learn more:
- <a href="https://cloud.ibm.com/apidocs/assistant-v2" target="_blank" rel="noopener noreferrer">Watson Assistant API reference</a>
- <a href="https://github.com/watson-developer-cloud/python-sdk" target="_blank" rel="noopener noreferrer">Watson Assistant Python SDK</a>

### Authors
Laurent Vincent.

Copyright &copy; IBM Corp. 2018. This notebook and its source code are released under the terms of the MIT License.